In [1]:
import numpy as np
import cv2
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout
from keras.models  import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import random,os,glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [18]:
dir_path = 'Garbage classification/Garbage classification'

In [19]:
img_list = glob.glob(os.path.join(dir_path, '*/*.jpg'))

In [57]:
len(img_list)

2527

In [21]:
train=ImageDataGenerator(horizontal_flip=True, vertical_flip=True,validation_split=0.1,rescale=1./255,
                         shear_range = 0.1,zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)
test=ImageDataGenerator(rescale=1/255,validation_split=0.1)
train_generator=train.flow_from_directory(dir_path,target_size=(300,300),batch_size=32,
                                          class_mode='categorical',subset='training')
test_generator=test.flow_from_directory(dir_path,target_size=(300,300),batch_size=32,
                                        class_mode='categorical',subset='validation')
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [22]:
model=Sequential()
    
model.add(Conv2D(32,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(6,activation='softmax'))

filepath="trained_model.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 43808)             0         
__________

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [25]:
model.fit_generator(train_generator, epochs=100, steps_per_epoch=2276//32,validation_data=test_generator,
                    validation_steps=251//32,callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





71/71 [==============================] - 131s 2s/step - loss: 1.7401 - acc: 0.2786 - val_loss: 1.6651 - val_acc: 0.2411

Epoch 00001: val_acc improved from -inf to 0.24107, saving model to trained_model.h5
Epoch 2/100
71/71 [==============================] - 129s 2s/step - loss: 1.4888 - acc: 0.3825 - val_loss: 1.7184 - val_acc: 0.3014

Epoch 00002: val_acc improved from 0.24107 to 0.30137, saving model to trained_model.h5
Epoch 3/100
71/71 [==============================] - 132s 2s/step - loss: 1.3869 - acc: 0.4313 - val_loss: 1.3627 - val_acc: 0.4840

Epoch 00003: val_acc improved from 0.30137 to 0.48402, saving model to trained_model.h5
Epoch 4/100
71/71 [==============================] - 134s 2s/step - loss: 1.2677 - acc: 0.4991 - val_loss: 1.2348 - val_acc: 0.4977

Epoch 00004: val_acc improved from 0.48402 to 0.49772, saving model to trained_model.h5
Epoch 5/100
71/71

71/71 [==============================] - 140s 2s/step - loss: 0.6922 - acc: 0.7421 - val_loss: 1.1690 - val_acc: 0.5936

Epoch 00032: val_acc did not improve from 0.68950
Epoch 33/100
71/71 [==============================] - 135s 2s/step - loss: 0.6897 - acc: 0.7478 - val_loss: 1.0764 - val_acc: 0.6473

Epoch 00033: val_acc did not improve from 0.68950
Epoch 34/100
71/71 [==============================] - 138s 2s/step - loss: 0.6590 - acc: 0.7571 - val_loss: 0.8819 - val_acc: 0.6849

Epoch 00034: val_acc did not improve from 0.68950
Epoch 35/100
71/71 [==============================] - 143s 2s/step - loss: 0.6246 - acc: 0.7680 - val_loss: 1.0218 - val_acc: 0.6301

Epoch 00035: val_acc did not improve from 0.68950
Epoch 36/100
71/71 [==============================] - 139s 2s/step - loss: 0.7281 - acc: 0.7469 - val_loss: 0.9028 - val_acc: 0.7078

Epoch 00036: val_acc improved from 0.68950 to 0.70776, saving model to trained_model.h5
Epoch 37/100
71/71 [==============================] - 1

In [26]:
model.save('garbage.model')

In [52]:
model = tf.keras.models.load_model("garbage.model")

In [87]:
CATEGORIES = ['cardboard','glass','metal','paper','plastic','trash']

In [95]:
predict_datagen = ImageDataGenerator(validation_split=0.1,rescale=1./255)

img1 = cv2.imread('plas.jfif')
img1 = cv2.resize(img1, (300, 300))
img1 = img1.reshape(1, 300, 300, -1)
print(img1.shape)

(1, 300, 300, 3)


In [97]:
prediction = model.predict(img1)
prediction

array([[1., 0., 0., 0., 0., 0.]], dtype=float32)